In [36]:
import pandas as pd

## Open Layoff and Non-Layoff Data

In [42]:
layoff_df = pd.read_csv("layoff_tech_companies.csv", index_col=False).drop(columns=["Unnamed: 0", "Index"])
non_layoff_df = pd.read_csv("non_layoff_stats.csv").drop(columns=["Unnamed: 0"])

## Add Headcount Change Feature

In [46]:
import pickle
# load layoff headcounts
with open("layoff_headcounts.pickle", "rb") as f:
    layoff_headcount_dict = pickle.load(f)
# load non-layoff headcounts
with open("non_layoff_headcounts.pickle", "rb") as f:
    non_layoff_headcount_dict = pickle.load(f)

In [47]:
# for each layoff ticker, find the headcount change in the previous 3 years
layoff_df["Ticker"].map(layoff_headcount_dict)
year_window = 3
for ticker, year in zip(layoff_df["Ticker"], layoff_df["Year"]):
    
    ticker_headcounts = layoff_headcount_dict[ticker]
    year_int = int(year)
    year_range = range(year_int - year_window, year_int)
    headcounts = ticker_headcounts[ticker_headcounts["Year"].isin(year_range)]["Number of Employees"]
    # headcounts = ticker_headcounts.loc[year_range, "Number of Employees"]
    headcounts = headcounts.dropna()
    # find headcount growth rate
    if len(headcounts) == 0:
        growth_rate = 0
    else:
        first_year = headcounts.iloc[-1]

        last_year = headcounts.iloc[0]
        # print(first_year, last_year)
        growth_rate = ((last_year - first_year) / first_year) * 100
    layoff_df.loc[layoff_df["Ticker"] == ticker, "Headcount Growth Rate"] = growth_rate
    # print(ticker, year, growth_rate)


In [48]:
# for each non-layoff ticker, find the headcount change in the previous 3 years
year_window = 3
start_year = 2023
for ticker in non_layoff_df["Ticker"]:
    ticker_headcounts = non_layoff_headcount_dict[ticker]
    year_int = int(start_year)
    year_range = range(year_int - year_window, year_int)
    headcounts = ticker_headcounts[ticker_headcounts["Year"].isin(year_range)]["Number of Employees"]
    # headcounts = ticker_headcounts.loc[year_range, "Number of Employees"]
    headcounts = headcounts.dropna()
    # find headcount growth rate
    if len(headcounts) == 0:
        growth_rate = 0
    else:
        first_year = headcounts.iloc[-1]

        last_year = headcounts.iloc[0]
        # print(first_year, last_year)
        growth_rate = ((last_year - first_year) / first_year) * 100
    non_layoff_df.loc[non_layoff_df["Ticker"] == ticker, "Headcount Growth Rate"] = growth_rate

## Filter out the columns that are not needed

In [49]:
def numeric_columns(df):
    numeric_columns = set()
    for col in df.columns:
        numeric = pd.to_numeric(df[col], errors="coerce")
        can_convert = ~numeric.isnull().all()
        if can_convert:
            numeric_columns.add(col)
    return numeric_columns

layoff_columns_set = numeric_columns(layoff_df)
non_layoff_columns_set = numeric_columns(non_layoff_df)
common_columns = layoff_columns_set.intersection(non_layoff_columns_set)

In [50]:
# preserve tickers
layoff_tickers = layoff_df["Ticker"]
non_layoff_tickers = non_layoff_df["Ticker"]
total_tickers = pd.concat([layoff_tickers, non_layoff_tickers], ignore_index=True).drop_duplicates()
common_columns.add("Ticker")

In [51]:
# filter out columns that are not in common_columns

layoff_numeric = layoff_df.loc[:, list(common_columns)]
layoff_numeric.fillna(0, inplace=True)
layoff_numeric["Layoffs"] = 1
non_layoff_numeric = non_layoff_df[list(common_columns)]
non_layoff_numeric.fillna(0, inplace=True)
non_layoff_numeric["Layoffs"] = 0

C:\Users\Sai Ramesh\AppData\Local\Temp\ipykernel_13996\3620432328.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_layoff_numeric.fillna(0, inplace=True)
C:\Users\Sai Ramesh\AppData\Local\Temp\ipykernel_13996\3620432328.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_layoff_numeric["Layoffs"] = 0


In [52]:
# combine the two dataframes
total_df = pd.concat([layoff_numeric, non_layoff_numeric])
# shuffle the dataframe to avoid bias
total_df = total_df.sample(frac=1, random_state=42).reset_index(drop=True)

## Add Categorical Feature

In [53]:
import financedatabase as fd
# Initialize the Equities database
equities = fd.Equities()

In [54]:
equities_df = equities.select()

In [55]:
ticker_list = total_df["Ticker"].tolist()

In [56]:
market_cap_values = equities_df.loc[ticker_list, :]["market_cap"].tolist()

In [57]:
# apply one hot encoding to categorical columns
total_df["Market Cap"] = market_cap_values
total_df = pd.get_dummies(total_df, columns=["Market Cap"])

In [58]:
total_df

,priceSalesRatio,bookValuePerShare,grossProfitGrowth,totalOtherIncomeExpensesNet,inventoryGrowth,averagePayables,cashAtEndOfPeriod,pfcfRatio,otherExpenses,Headcount Growth Rate,...,cashRatio,daysOfPayablesOutstanding,grossProfitRatio,Layoffs,Market Cap_Large Cap,Market Cap_Mega Cap,Market Cap_Micro Cap,Market Cap_Mid Cap,Market Cap_Nano Cap,Market Cap_Small Cap
0,0.923419,10.915670,-0.153057,-7.307000e+06,0.392614,8.802750e+07,1.091303e+08,-14.287295,1.941900e+07,0.000000,...,0.513723,81.243042,0.331188,0,0,0,0,0,0,1
1,2.753718,56.650163,0.233252,-1.010667e+08,0.309747,1.786328e+08,8.278667e+08,16.900002,2.032000e+08,87.931034,...,1.559581,49.455494,0.451500,0,0,0,0,1,0,0
2,6.747642,30.446408,4.754562,-4.962300e+07,0.000000,2.593200e+07,1.768066e+10,4.969610,6.303080e+08,198.638911,...,0.623825,11.466811,0.838264,1,1,0,0,0,0,0
3,0.385808,124.678729,-0.020037,-8.950000e+08,0.185543,3.520417e+10,1.622600e+10,20.152112,3.973333e+08,8.333333,...,0.259653,126.183396,0.171405,0,0,0,0,1,0,0
4,12.489858,0.181337,0.445312,1.665267e+07,1.049301,1.485033e+07,6.543333e+06,25844.683072,8.618333e+06,19.799139,...,0.247727,40.857947,0.389465,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743,15.001349,11.909725,0.398543,-2.597000e+06,0.000000,8.681500e+06,8.839900e+07,-166.383627,0.000000e+00,19.379015,...,0.504737,37.236139,0.593183,1,0,0,0,1,0,0
744,12.253327,16.298458,0.602682,-5.407300e+07,0.000000,3.805950e+07,1.692632e+09,-49.985845,3.125780e+08,79.148472,...,13.556270,63.913094,0.621065,1,0,0,0,1,0,0
745,6.150032,15.977752,0.323007,1.519867e+07,0.000000,4.645300e+07,6.381583e+08,-57.226773,2.210000e+05,21.739130,...,3.721951,136.043062,0.664945,0,0,0,0,0,0,1
746,35.299020,28.877760,0.938735,-1.202367e+07,0.000000,7.714500e+06,2.314928e+09,-560.050303,1.521000e+07,267.152104,...,0.343201,42.590522,0.755449,0,0,0,0,1,0,0


In [59]:
layoff_df

,period,revenuePerShare,netIncomePerShare,operatingCashFlowPerShare,freeCashFlowPerShare,cashPerShare,bookValuePerShare,tangibleBookValuePerShare,shareholdersEquityPerShare,interestDebtPerShare,...,Funds_Raised,Stage,Date_Added,Country,Percentage,List_of_Employees_Laid_Off,Month,Year,Ticker,Headcount Growth Rate
0,FY,32.733348,3.849057,1.781567,1.249147,7.935144,14.325357,11.148507,14.325357,0.257652,...,NaN,Post-IPO,2023-03-23 16:47:11,Switzerland,NaN,Unknown,2023-03,2023.0,LOGI,24.242424
1,FY,5.599994,-0.528370,1.028252,0.793428,0.769943,19.705087,-3.077890,19.705087,6.106278,...,NaN,Post-IPO,2023-03-21 23:29:14,United States,0.04,Unknown,2023-03,2023.0,MRVL,19.456773
2,FY,74.467678,2.246732,0.021957,0.017731,26.450163,14.571844,-38.736979,14.571844,43.587878,...,3300.0,Post-IPO,2023-03-22 23:59:38,United States,NaN,Unknown,2023-03,2023.0,EXPE,-13.612565
3,FY,50.444892,-0.267151,4.588478,-1.657964,6.872706,14.333399,12.342232,14.333399,13.984199,...,108.0,Post-IPO,2023-03-20 15:07:51,United States,NaN,Unknown,2023-03,2023.0,AMZN,101.503759
4,FY,31.844303,2.254025,3.968584,0.950543,15.289061,42.085799,23.639859,42.085799,24.574589,...,NaN,Post-IPO,2023-03-17 04:11:53,United States,NaN,Unknown,2023-03,2023.0,COHR,2.999695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,FY,21.610733,3.551107,5.400341,4.626917,8.091993,18.506814,7.615843,18.506814,7.904600,...,216.0,Post-IPO,2022-05-26 03:02:05,United States,NaN,Unknown,2022-05,2022.0,PYPL,33.189655
138,FY,72.150900,-0.760135,-14.605856,-17.742117,2.269144,1.722973,2.882883,1.722973,33.502252,...,1600.0,Post-IPO,2022-05-10 15:35:42,United States,0.12,Unknown,2022-05,2022.0,CVNA,186.728564
139,FY,43.414415,-5.057034,-7.753543,-8.140928,15.438220,12.477836,9.927934,12.477836,15.840739,...,1300.0,Post-IPO,2022-05-10 02:49:25,United States,0.14,Unknown,2022-05,2022.0,VRM,125.875000
140,FY,16.417323,-2.842520,-7.488189,-7.503937,9.125984,6.440945,6.440945,6.440945,0.251969,...,1600.0,Post-IPO,2022-05-14 01:20:14,United States,0.15,Unknown,2022-03,2022.0,WISH,47.101449


In [60]:
# drop duplicates
total_df = total_df.drop_duplicates(subset=["Ticker"])

In [61]:
# filter out invalid tickers

import yfinance as yf
with open("expired_tickers.pickle", "rb") as f:
    expired_tickers = pickle.load(f)
for i, ticker in enumerate(total_df["Ticker"], 1):
    if ticker not in expired_tickers:
        try:
            ticker_obj = yf.Ticker(ticker)
            ticker_obj.info
        except:
            expired_tickers.add(ticker)
    print(f"Finished {i} tickers out of {len(total_df)}", end="\r")

In [62]:
# save invalid tickers
with open("expired_tickers.pickle", "wb") as f:
    pickle.dump(expired_tickers, f)
#invalid_tickers

In [64]:
# filter out invalid tickers
total_df = total_df[~total_df["Ticker"].isin(expired_tickers)]

In [66]:
total_df

,priceSalesRatio,bookValuePerShare,grossProfitGrowth,totalOtherIncomeExpensesNet,inventoryGrowth,averagePayables,cashAtEndOfPeriod,pfcfRatio,otherExpenses,Headcount Growth Rate,...,cashRatio,daysOfPayablesOutstanding,grossProfitRatio,Layoffs,Market Cap_Large Cap,Market Cap_Mega Cap,Market Cap_Micro Cap,Market Cap_Mid Cap,Market Cap_Nano Cap,Market Cap_Small Cap
1,2.753718,56.650163,0.233252,-1.010667e+08,0.309747,1.786328e+08,8.278667e+08,16.900002,2.032000e+08,87.931034,...,1.559581,49.455494,0.451500,0,0,0,0,1,0,0
2,6.747642,30.446408,4.754562,-4.962300e+07,0.000000,2.593200e+07,1.768066e+10,4.969610,6.303080e+08,198.638911,...,0.623825,11.466811,0.838264,1,1,0,0,0,0,0
3,0.385808,124.678729,-0.020037,-8.950000e+08,0.185543,3.520417e+10,1.622600e+10,20.152112,3.973333e+08,8.333333,...,0.259653,126.183396,0.171405,0,0,0,0,1,0,0
4,12.489858,0.181337,0.445312,1.665267e+07,1.049301,1.485033e+07,6.543333e+06,25844.683072,8.618333e+06,19.799139,...,0.247727,40.857947,0.389465,0,0,0,0,1,0,0
5,0.340962,262.006000,0.356486,1.054285e+08,0.436971,6.316611e+09,7.815326e+09,-2.256531,-5.823669e+08,27.375723,...,0.194829,60.706969,0.162191,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
740,1.879526,2738.079615,0.000850,7.560000e+08,0.055448,8.103570e+10,1.972573e+11,33.911015,9.723333e+08,3.620653,...,0.955211,68.549517,0.452561,0,1,0,0,0,0,0
742,10.975483,49.412363,0.154069,-1.543133e+07,0.000000,4.247590e+07,3.622170e+08,49.256035,4.262467e+07,30.057803,...,0.482336,21.817797,0.451724,0,1,0,0,0,0,0
743,15.001349,11.909725,0.398543,-2.597000e+06,0.000000,8.681500e+06,8.839900e+07,-166.383627,0.000000e+00,19.379015,...,0.504737,37.236139,0.593183,1,0,0,0,1,0,0
745,6.150032,15.977752,0.323007,1.519867e+07,0.000000,4.645300e+07,6.381583e+08,-57.226773,2.210000e+05,21.739130,...,3.721951,136.043062,0.664945,0,0,0,0,0,0,1


## Save the data

In [67]:
# save the dataframe to csv
total_df.to_csv("total_df.csv", index=False)

In [68]:
total_df.columns

Index(['priceSalesRatio', 'bookValuePerShare', 'grossProfitGrowth',
       'totalOtherIncomeExpensesNet', 'inventoryGrowth', 'averagePayables',
       'cashAtEndOfPeriod', 'pfcfRatio', 'otherExpenses',
       'Headcount Growth Rate',
       ...
       'cashRatio', 'daysOfPayablesOutstanding', 'grossProfitRatio', 'Layoffs',
       'Market Cap_Large Cap', 'Market Cap_Mega Cap', 'Market Cap_Micro Cap',
       'Market Cap_Mid Cap', 'Market Cap_Nano Cap', 'Market Cap_Small Cap'],
      dtype='object', length=201)